<a href="https://colab.research.google.com/github/AishwaryaKaranth/final-year-project/blob/master/All%20Models/Model_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install git+https://github.com/keras-team/keras-preprocessing.git

  Cloning https://github.com/keras-team/keras-preprocessing.git to /tmp/pip-req-build-pm1gvyi5
  Running command git clone -q https://github.com/keras-team/keras-preprocessing.git /tmp/pip-req-build-pm1gvyi5
  Created wheel for Keras-Preprocessing: filename=Keras_Preprocessing-1.1.2-cp37-none-any.whl size=43617 sha256=bf67215dd35d2045f5b6f127a409f176b9c6272f07f24ee212d9f4feb2cb2c65
  Stored in directory: /tmp/pip-ephem-wheel-cache-saxdguo3/wheels/03/a0/39/171f6040d36f36c71168dc69afa81334351b20955dc36ce932
Successfully built Keras-Preprocessing


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip "/content/drive/MyDrive/Dataset/train/images_training_rev1.zip" -d "/content/drive/MyDrive/Dataset/"

In [ ]:
import keras
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda, Reshape
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.optimizers import RMSprop
import pandas as pd
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import load_img,img_to_array
from keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from keras_preprocessing.image import ImageDataGenerator

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
!unzip '/content/training_solutions_rev1.zip'

In [ ]:
df=pd.read_csv('training_solutions_rev1.csv')

In [ ]:
df

In [ ]:

classes = [
    'Class1.1', 'Class1.2', 'Class1.3', 'Class2.1', 'Class2.2', 'Class3.1','Class3.2', 'Class4.1', 'Class4.2', 'Class5.1', 'Class5.2', 'Class5.3','Class5.4', 'Class6.1', 'Class6.2', 'Class7.1', 'Class7.2', 'Class7.3',
    'Class8.1', 'Class8.2', 'Class8.3', 'Class8.4', 'Class8.5', 'Class8.6','Class8.7', 'Class9.1', 'Class9.2', 'Class9.3', 'Class10.1', 'Class10.2','Class10.3', 'Class11.1', 'Class11.2', 'Class11.3', 'Class11.4',
    'Class11.5', 'Class11.6'
]

In [ ]:
def apply_jpg(f):
  return f+'.jpg'

In [ ]:
df['GalaxyID']=df['GalaxyID'].astype(str).apply(apply_jpg)

In [ ]:
d=ImageDataGenerator(
    fill_mode='nearest',
    cval=0,
    rescale=1. / 255,
    rotation_range=90,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.02
)

In [ ]:
train_generator = d.flow_from_dataframe(
    dataframe=df,
    directory="/content/drive/MyDrive/Dataset/he1",
    x_col="GalaxyID",
    y_col=classes,
    subset="training",
    batch_size=64,
    seed=123,
    shuffle=True,
    class_mode="raw",
    target_size=(224, 224))

In [ ]:
valid_generator = d.flow_from_dataframe(
    dataframe=df,
    directory="/content/drive/MyDrive/Dataset/he1",
    x_col="GalaxyID",
    y_col=classes,
    subset="validation",
    batch_size=64,
    seed=123,
    shuffle=True,
    class_mode="raw",
    target_size=(224, 224))

In [ ]:
train_size=train_generator.n // train_generator.batch_size
valid_size=valid_generator.n // valid_generator.batch_size

In [ ]:
train_size, valid_size

In [ ]:
def ConvLayers(layers, model, filters):
  for i in range(layers):
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(filters, 3,3,activation='relu'))
  model.add(MaxPooling2D((2,2),strides=(1,1),padding="same"))

In [ ]:
def FCLayers(model):
  model.add(Dense(4096, activation='relu'))
  model.add(Dropout(0.5))

In [ ]:
def VGG16_arc():
  model=Sequential()
  ConvLayers(2,model,64)
  ConvLayers(2,model,128)
  ConvLayers(3,model,256)
  ConvLayers(3,model,512)
  ConvLayers(3,model,512)
  model.add(Flatten())
  FCLayers(model)
  FCLayers(model)
  model.add(Dense(37,activation='sigmoid'))
  #model.add(Dense(3,activation='softmax'))
  return model

In [ ]:
optimizer = keras.optimizers.Adam(lr=0.001, decay=5e-4)
model=VGG16_arc()
model.compile(loss='mean_squared_error',optimizer=optimizer,metrics=['accuracy'])

In [ ]:
model.build(input_shape=(None,224,224,3))
model.summary()

In [ ]:
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping



checkpoint_path='weights.hdf5'
checkpoint_callback=ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, monitor='accuracy',mode='max',save_freq='epoch',save_best_only=True)

In [ ]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=train_size,
    validation_data=valid_generator,
    validation_steps=valid_size,
    epochs=10,
    callbacks=[checkpoint_callback])